In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [2]:
file="50ksample.csv"

In [3]:
data_df=pd.read_csv(file)
data_df.head()

C:\Users\Huda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1514734,2b7b688e4a4315bc4f21f6849d03ab66,win8defender,1.1.15200.1,4.11.15063.0,1.275.828.0,0,7.0,0,NaN,...,33968.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
1,2926031,53f3fd511ba64acaec4e0c8559143301,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,...,19951.0,0,0.0,0.0,0,0,0.0,1.0,5.0,0
2,7189617,ce48198690d1e4b437ffb60bfcb9e8df,win8defender,1.1.15100.1,4.18.1807.18075,1.273.516.0,0,7.0,0,NaN,...,33066.0,0,NaN,0.0,0,0,0.0,1.0,13.0,1
3,6264912,b3cfee470ee30c729d3a33f0017a8b51,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,...,12303.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
4,3321923,5f4f1fa2419439ff865d42a2a06ac8b5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1486.0,0,7.0,0,NaN,...,4357.0,0,NaN,0.0,0,1,0.0,0.0,3.0,1


In [4]:
data_df=data_df[["EngineVersion","AVProductStatesIdentifier","AVProductsEnabled","Processor","SmartScreen","Census_MDC2FormFactor","Census_OSVersion","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","HasDetections"]]

data_df.head()


,EngineVersion,AVProductStatesIdentifier,AVProductsEnabled,Processor,SmartScreen,Census_MDC2FormFactor,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,53447.0,1.0,x64,NaN,Desktop,10.0.15063.0,0.0,0.0,0
1,1.1.15200.1,53447.0,1.0,x64,RequireAdmin,Desktop,10.0.16299.125,0.0,1.0,0
2,1.1.15100.1,53447.0,1.0,x64,NaN,Notebook,10.0.15063.1088,0.0,1.0,1
3,1.1.14600.4,47238.0,1.0,x64,NaN,Notebook,10.0.17134.112,0.0,0.0,0
4,1.1.15200.1,53447.0,1.0,x64,NaN,Desktop,10.0.17134.254,0.0,0.0,1


## Iteration 1

Variable choices:



Drop that have nulls in AVProductStatesIdentifier

AVProductsEnabled fill nulls with 0
Browser Identtifier needs to be filled wiht 0
SmartScreen: clean up off and Off, on and On, “&#x02” and “&#x01" will have own columns., null should go to off bucket
Census_IsAlwaysConnectedCapable: replace with 0
Wdft_IsGamer: replace with 0



In [5]:
### filling and dropping nulls
data_df[["AVProductsEnabled","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off")
## This drops the nulls remaining in AVProductsStatesIdentifier
data_df.dropna(how='any',inplace=True)


C:\Users\Huda\Anaconda3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47734 entries, 0 to 49999
Data columns (total 10 columns):
EngineVersion                              47734 non-null object
AVProductStatesIdentifier                  47734 non-null float64
AVProductsEnabled                          47734 non-null float64
Processor                                  47734 non-null object
SmartScreen                                47734 non-null object
Census_MDC2FormFactor                      47734 non-null object
Census_OSVersion                           47734 non-null object
Census_IsAlwaysOnAlwaysConnectedCapable    47734 non-null float64
Wdft_IsGamer                               47734 non-null float64
HasDetections                              47734 non-null int64
dtypes: float64(4), int64(1), object(5)
memory usage: 4.0+ MB


In [7]:
y=data_df["HasDetections"]
len(y)

47734

In [8]:
X=pd.get_dummies(data_df)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,AVProductStatesIdentifier,AVProductsEnabled,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,EngineVersion_1.1.10302.0,EngineVersion_1.1.11701.0,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,EngineVersion_1.1.12902.0,...,Census_OSVersion_10.0.17746.1000,Census_OSVersion_10.0.17747.1000,Census_OSVersion_10.0.17751.1,Census_OSVersion_10.0.17754.1,Census_OSVersion_10.0.17755.1,Census_OSVersion_10.0.17758.1,Census_OSVersion_10.0.17758.4,Census_OSVersion_10.0.17760.1,Census_OSVersion_10.0.17763.1,Census_OSVersion_10.0.18234.1000
0,53447.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,53447.0,1.0,0.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,53447.0,1.0,0.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,47238.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53447.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [10]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [11]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )



Training: 0.7793016759776537
Testing: 0.5933467404055639


In [13]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.270950230518752, 'AVProductStatesIdentifier'),
 (0.06682707003875459, 'SmartScreen_ExistsNotSet'),
 (0.03176108052095107, 'Wdft_IsGamer'),
 (0.02624973892749944, 'SmartScreen_RequireAdmin'),
 (0.021831767106072084, 'SmartScreen_off'),
 (0.02099003583485794, 'Census_MDC2FormFactor_Notebook'),
 (0.01921837483269523, 'EngineVersion_1.1.15100.1'),
 (0.017140150549915186, 'Census_MDC2FormFactor_Desktop'),
 (0.015927758008994156, 'EngineVersion_1.1.15200.1'),
 (0.011597737931674547, 'Census_OSVersion_10.0.16299.431'),
 (0.011551595340894323, 'Census_OSVersion_10.0.17134.228'),
 (0.011339244685388853, 'Census_MDC2FormFactor_Convertible'),
 (0.01129747269934861, 'AVProductsEnabled'),
 (0.010109661206237748, 'Census_OSVersion_10.0.17134.165'),
 (0.009998566204971526, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.009954473900945315, 'Census_OSVersion_10.0.17134.112'),
 (0.009482612328917214, 'Census_MDC2FormFactor_AllInOne'),
 (0.008728404095898899, 'Census_OSVersion_10.0.16299.371'),
 (0.

In [14]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

              precision    recall  f1-score   support

           0       0.60      0.55      0.57      5912
           1       0.59      0.63      0.61      6022

   micro avg       0.59      0.59      0.59     11934
   macro avg       0.59      0.59      0.59     11934
weighted avg       0.59      0.59      0.59     11934



## Iteration 2

Some of the previous values did not one hot encode correctly.

Will write a script to encode them and try with that

In [15]:
iter2=data_df.copy()

In [16]:
iter2.head()

,EngineVersion,AVProductStatesIdentifier,AVProductsEnabled,Processor,SmartScreen,Census_MDC2FormFactor,Census_OSVersion,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,53447.0,1.0,x64,off,Desktop,10.0.15063.0,0.0,0.0,0
1,1.1.15200.1,53447.0,1.0,x64,RequireAdmin,Desktop,10.0.16299.125,0.0,1.0,0
2,1.1.15100.1,53447.0,1.0,x64,off,Notebook,10.0.15063.1088,0.0,1.0,1
3,1.1.14600.4,47238.0,1.0,x64,off,Notebook,10.0.17134.112,0.0,0.0,0
4,1.1.15200.1,53447.0,1.0,x64,off,Desktop,10.0.17134.254,0.0,0.0,1


In [17]:
iter2['AVProductStatesIdentifier'] = iter2['AVProductStatesIdentifier'].astype(str)
iter2['AVProductsEnabled'] = iter2['AVProductsEnabled'].astype(str)


In [18]:
X=pd.get_dummies(iter2)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,EngineVersion_1.1.10302.0,EngineVersion_1.1.11701.0,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,EngineVersion_1.1.12902.0,EngineVersion_1.1.13000.0,EngineVersion_1.1.13103.0,...,Census_OSVersion_10.0.17746.1000,Census_OSVersion_10.0.17747.1000,Census_OSVersion_10.0.17751.1,Census_OSVersion_10.0.17754.1,Census_OSVersion_10.0.17755.1,Census_OSVersion_10.0.17758.1,Census_OSVersion_10.0.17758.4,Census_OSVersion_10.0.17760.1,Census_OSVersion_10.0.17763.1,Census_OSVersion_10.0.18234.1000
0,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [20]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [21]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.7793016759776537
Testing: 0.6028154851684263


In [22]:
for i  in range(len(dummy_columns)):
    dummy_columns[i]=str(dummy_columns[i])

In [23]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.06528438704014038, 'SmartScreen_ExistsNotSet'),
 (0.03780243221374094, 'AVProductStatesIdentifier_53447.0'),
 (0.03701265247717966, 'Wdft_IsGamer'),
 (0.03056011098231916, 'SmartScreen_RequireAdmin'),
 (0.02208051631091246, 'Census_MDC2FormFactor_Notebook'),
 (0.021425036336771223, 'SmartScreen_off'),
 (0.021374654817350024, 'EngineVersion_1.1.15100.1'),
 (0.018009524579589698, 'EngineVersion_1.1.15200.1'),
 (0.0169541507124263, 'Census_MDC2FormFactor_Desktop'),
 (0.013598277685993261, 'AVProductStatesIdentifier_7945.0'),
 (0.013207350373267621, 'Census_OSVersion_10.0.16299.431'),
 (0.012871998197052682, 'Census_OSVersion_10.0.17134.228'),
 (0.011833273299082302, 'Census_OSVersion_10.0.17134.165'),
 (0.011581108784579524, 'Census_MDC2FormFactor_Convertible'),
 (0.010459452885950751, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.010458791369424165, 'Census_OSVersion_10.0.17134.112'),
 (0.00979847612339879, 'Census_OSVersion_10.0.16299.371'),
 (0.009445933891154208, 'Census_MDC2For

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59      5912
           1       0.60      0.63      0.61      6022

   micro avg       0.60      0.60      0.60     11934
   macro avg       0.60      0.60      0.60     11934
weighted avg       0.60      0.60      0.60     11934



## Iteration 3

We have improvement.

I'm goinig to attempt to just up the estimators

In [26]:
rf = RandomForestClassifier(n_estimators=400)
rf = rf.fit(X_train, y_train)

In [27]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.7793016759776537
Testing: 0.6023965141612201


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59      5912
           1       0.60      0.63      0.61      6022

   micro avg       0.60      0.60      0.60     11934
   macro avg       0.60      0.60      0.60     11934
weighted avg       0.60      0.60      0.60     11934

